In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Docker settings:

### 1. Pulling (download) the image dockerfile for pytorch (similar with tensorflow)

<font color=orange>
NOTICE:

Currently the pytorch image for development version Supporting Cuda 10
4 GB too large for the login node on OSC

For issues with larger file: See  below


https://www.osc.edu/resources/available_software/software_list/singularity

You might encounter an error while pulling a large Docker image:

    [owens-login01]$ singularity pull docker://qimme2/core
    FATAL: Unable to pull docker://qiime2/core While running mksquashfs: signal: killed
    The process could killed because the image is cached in the home directory which is a slower file system or the image size might exceed a single file size limit.

    The solution is to use other file systems like /fs/scratch and $TMPDIR for caches, squashfs temp files and download

    [owens-login01]$ qsub -I -l nodes=1:ppn=1
    [o0001]$ cd $TMPDIR
    [o0001]$ export SINGULARITY_CACHEDIR=$TMPDIR
    [o0001]$ export SINGULARITY_TMPDIR=$TMPDIR
    [o0001]$ singularity pull docker://qiime2/core:2019.1
    [o0001]$ cp qiime2_core_2019.1.sif /where/to/keep/image/  
    
**NOTE:**

- DO not try to run .sif file directly from the \$TMPDIR , Permission Denied for \$TMPDIR

### 2. Trial nvcc and cuda availability 

<font color=orange>
Before you jump into the gpu version of the pytorch, 
Need to follow the following procedure to first use a GPU node, and test using torch.cuda


        If you have a GPU-enabled container you can easily run it on Owens or Pitzer just by adding the --nv flag to the singularity exec or run command.  The example below comes from the exec command section of Singularity User Guide.  It runs a TensorFlow example using a GPU on Owens.  (Output has been omitted from the example for brevity.)

        [owens-login01]$ qsub -I -l nodes=1:ppn=28:gpus=1
        ...
        [o0756]$ cd $PBS_O_WORKDIR
        [o0756]$

         git clone https://github.com/tensorflow/models.git

        [o0756]$

         singularity exec --nv ~/Dockers/yourDockerFile.sif python

<font color=orange>
When you use the python interactive
Test the pytorch by:

import torch
torch.cuda.is_available()

Test the tensorflow by:

import tensorflow as tf
tf.test.is_gpu_available()
s

### 3. Modify the docker

<font color=orange>
    
Mainly two ways of modifying the dockers:

1) modify the source code, i.e. dockerfile

2) lauch an established image as a container and use the commit to modify it:

    docker run -it original_image_name bash
    exit   (after exiting, the container won't be deleted right away)
    docker container ls --all     (through --all option, you can still track the last container)
    docker commit container-ID To_be_modified_image-name


Helpful link:

https://bobcares.com/blog/edit-docker-image/

### 4. NOTICE

In [ ]:
'''
1) as for the environmental variable:
For the implementation in OSC: 
The system wide has the highest privileges over the environmental viriable settings:
Load or unload specific modules can change the environmental variables:

Therefore:
a) unload modules if we do not need them

b) Do not set PYTHONPATH if you use the docker image otherwise the python packages installed inside
   the docker images may not work properly

2)  if you need specific environmental settings:
    (A) Ease but ugly way:
        For general use: like CPATH, PATH, LD_LIBRARY_PATH
        set them in ~/.bash_profile : user wide environment
        Those variables may be a path relative to the docker image, which is fine
        system will ignore it if it did not find it from left to the right
        For specific use:
        you can set them in the ENV in the dockerfile for a specific docker image
        However, you should make sure the variables are not in conflict with the system wide variables
        Otherwise, it will always be overriden by the higher privilege settings
    (B) More wise and flexible way:
        We need this probably because OSC has higher privileges managment of the environment variables
        1) Since we use the singulairty to build the .sif image from an established docker image:
           Use the entrypoint with export to set a run-time env for the container
           The ENV set by the Dockerfile is shadowed especially the variable names conflict with
           some system-wide env
           However, this run time setting will always work and will append to the existing env from h
           System-wide or user-wide anyway with higher privilege
           Notice: only the last entrypoint will be executed if you have multiple ones in Dockerfile
        2) When you build the singularity file from the local machine then scp to the OSC server:
           The same thing happens although you set the %environment section in the definition file
           Still the environment is set at runtime and appended to the higher privilege env
           
3) For CUDA especially:
   We have the pytorch with cuda image, no need to load this module
'''

### Interacting with Singularity on OSC

Use singularity shell script to open a shell script of the container

#### Use pip to install packages inside the dockerfile

Possible issues when we need to uninstall some python modules due to version issue

- Uninstall:

pip uninstall -y networkx

pip install -I networkx==2.3.0    # since 2.4 has some incompatible issues

